**Mount the Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/CSE6250'
import warnings
warnings.filterwarnings('ignore')

**Installation of Libraries**

In [2]:
!pip install --upgrade ipykernel
!pip install keras
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Import Statements**

In [3]:
import time
import csv
import string
import datetime
import numpy as np

In [4]:
from nltk import word_tokenize
from keras.layers import Conv2D
from keras.layers import Concatenate
from keras.models import Sequential, Model
from keras.optimizers import adam as adam_v2
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from gensim.models.keyedvectors import KeyedVectors
from keras.layers import Dense, Dropout, Flatten, Input, MaxPool2D


2023-04-19 23:20:42.107305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 23:20:46.365700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


**File Setup**

In [5]:
# will allow us to remove punctuation and clean up sentences
punctuations = list(string.punctuation)

# csv file: "User", "Post", "Label"
ip_txt_file = "data/500_Reddit_users_posts_labels.csv"  

# csv file: "User", "Features"
ip_feat_file = "data/External_Characteristic_Features.csv"            

# conceptnet embeddings
w2v_file = {'file': 'data/numberbatch-en.txt', 'is_binary': False}

#file used to save the model outputs with 300 features
# op_file = "data/Result_5-Label_Classification_300features.tsv"

#file used to save the model outputs with 314 features
op_file = "data/Result_5-Label_Classification.tsv"

severity_classes = {'Supportive': 0, 
                    'Indicator': 1, 
                    'Ideation': 2, 
                    'Behavior': 3, 
                    'Attempt': 4}

**Hyper-parameter Setup**

In [6]:
# system parameters across all models
sys_params = {'emb_dim': 300,
              'max_sent_len': 1500,
              'str_padd': '@PADD',
              'cross_val': 5}

# cnn parameters 
cnn_params = {'no_filters': 100,
              'kernels': [3, 4, 5],
              'channel': 1,
              'c_stride': (1, sys_params['emb_dim']),
              'pad': 'same',
              'ip_shape': (sys_params['max_sent_len'], sys_params['emb_dim'], 1),
              'c_activ': 'relu',
              'drop_rate': 0.3,
              'dense_1_unit': 128,
              'dense_2_unit': 128,
              'dense_activ': 'relu',
              'op_unit': 5,
              'op_activ': 'softmax',
              'l_rate': 0.005,
              'loss': 'categorical_crossentropy',
              'batch': 4,
              'epoch': 5,
              'verbose': 1}

# feature extraction for the last layer
intermediate_layer = 'flat_drop'

print ('\nSystem Parameters: ', sys_params)
print ('\nCNN Parameters: ', cnn_params)


System Parameters:  {'emb_dim': 300, 'max_sent_len': 1500, 'str_padd': '@PADD', 'cross_val': 5}

CNN Parameters:  {'no_filters': 100, 'kernels': [3, 4, 5], 'channel': 1, 'c_stride': (1, 300), 'pad': 'same', 'ip_shape': (1500, 300, 1), 'c_activ': 'relu', 'drop_rate': 0.3, 'dense_1_unit': 128, 'dense_2_unit': 128, 'dense_activ': 'relu', 'op_unit': 5, 'op_activ': 'softmax', 'l_rate': 0.005, 'loss': 'categorical_crossentropy', 'batch': 4, 'epoch': 5, 'verbose': 1}


**Read the input csv file**

In [7]:
def read_ip_file(ip_file):

    padd = sys_params['str_padd']
    max_len = sys_params['max_sent_len']

    x_data, y_data = [], []

    if ip_file:
        with open(ip_file,"r") as csv_file:

            # exclude the first line (header)
            csv_reader = csv.reader(csv_file, delimiter=',')
            next(csv_reader)

            # loop through each line
            for row in csv_reader:

                sent = row[1]

                # remove non-ascii characters
                printable = set(string.printable)

                # remove punctuation and non-ascii characters
                lst_tokens = [item.lower().strip("".join(punctuations)) for item in word_tokenize(sent) if
                              item not in punctuations and item not in printable]

                # strip the sentence if it exceeds the max length
                if len(lst_tokens) > max_len:
                    lst_tokens = lst_tokens[:max_len]

                # pad the sentence if the length is less than max length
                elif len(lst_tokens) < max_len:
                    for j in range(len(lst_tokens), max_len):
                        lst_tokens.append(padd)

                y_data.append(severity_classes[row[2].strip()])
                x_data.append(lst_tokens)

    return x_data, y_data

**Vectorizing input data using pretrained word2vec embedding**

In [8]:
def vectorize_data(lst_input):

    padd = sys_params['str_padd']
    wv_size = sys_params['emb_dim']

    # Load the pre-trained word2vec model
    w2v_model = KeyedVectors.load_word2vec_format(w2v_file['file'], binary=w2v_file['is_binary'])

    # Get the word2vec vocabulary
    vocab =w2v_model.key_to_index
    #
    padding_zeros = np.zeros(wv_size, dtype=np.float32)

    x_data = []

    # Loop through each sentence
    for sent in lst_input:
        emb = []
        for tok in sent:

            # Zero-padding for padded tokens
            if tok.lower() == padd:
                emb.append(list(padding_zeros))

            # Get the token embedding from the word2vec model
            elif tok.lower() in vocab.keys():
                emb.append(w2v_model[tok.lower()].astype(float).tolist())
            # Zero-padding for out-of-vocab tokens
            else:
                emb.append(list(padding_zeros))

        x_data.append(emb)

    del w2v_model, vocab

    return np.array(x_data)

**Preparing input data**

In [9]:
# Prepare the input data
def read_data(ip_file):

    # Read the input file
    x_data, y_data = read_ip_file(ip_file)

    # Vectorize the data
    x_data = vectorize_data(x_data)

    # # Reshape the data for CNN
    x_data = x_data.reshape(x_data.shape[0], x_data.shape[1], x_data.shape[2], 1)  # last argument 1 indicates #channel

    # Convert into numpy array
    x_data, y_data = np.array(x_data), np.array(y_data)

    return x_data, y_data

**Reading external features**

In [10]:
def read_external_features(raw_data, raw_features):

    user_ids, features = [], []

    # Read the user ids from raw_data csv and append them to "user_ids" list
    with open(raw_data) as file:
        for line in file:
            split = line.strip().split(',')
            user_ids.append(split[0])

    with open(raw_features) as csv_file:
        # Read the feature file
        csvreader = csv.reader(csv_file, delimiter=',')
        # Skip the header row
        header = next(csvreader)
        # Loop through each user feature row
        for row in csvreader:
            # Convert the feature score into float
            scores = [float(value) for value in row[1:]]
            # Append feature score list for each user to final features list
            features.append(scores)

    # Return numpy array of features
    return np.array(features)

**CNN and MLP Model**

In [11]:
def get_cnn_model():
    seq_len = sys_params['max_sent_len']
    emb_dim = sys_params['emb_dim']

    l_ip = Input(shape=(seq_len, emb_dim, 1), dtype='float32')
    lst_convfeat = []
    for filter in cnn_params['kernels']:
        l_conv = Conv2D(filters=cnn_params['no_filters'], kernel_size=(filter, emb_dim), strides=cnn_params['c_stride'],
                        padding=cnn_params['pad'], data_format='channels_last', input_shape=cnn_params['ip_shape'],
                        activation=cnn_params['c_activ'])(l_ip)
        l_pool = MaxPool2D(pool_size=(seq_len, 1))(l_conv)
        lst_convfeat.append(l_pool)

    l_concat = Concatenate(axis=1)(lst_convfeat)
    l_flat = Flatten()(l_concat)
    l_drop = Dropout(rate=cnn_params['drop_rate'], name='flat_drop')(l_flat)

    l_op = Dense(units=cnn_params['op_unit'], activation=cnn_params['op_activ'], name='cnn_op')(l_drop)

    final_model = Model(l_ip, l_op)
    final_model.compile(optimizer=adam_v2.Adam(learning_rate=cnn_params['l_rate']), loss=cnn_params['loss'], metrics=['accuracy'])    # 'categorical_crossentropy'

    return final_model

# Returns a MLP model for final classification
def get_mlp_model(ip_dim):

    mlp_model = Sequential()

    mlp_model.add(Dense(units=cnn_params['op_unit'], activation=cnn_params['op_activ'], name='classif_op',
                            input_dim=ip_dim))

    mlp_model.compile(optimizer=adam_v2.Adam(learning_rate=cnn_params['l_rate']), loss=cnn_params['loss'],
                          metrics=['accuracy'])
    return mlp_model

**Calculating Precision, Recall, F1 Score and Ordinal Error**

In [12]:
# Compute Precision, Recall, and F1-score
def get_prf1_score(y_true, y_pred):
    tp, fp, fn = 0.0, 0.0, 0.0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            tp += 1
        elif y_pred[i] > y_true[i]:
            fp += 1
        else:
            fn += 1
    if tp == 0:
        tp = 1.0
    if fp == 0:
        fp = 1.0
    if fn == 0:
        fn  = 1.0
    P = tp / (tp + fp)
    R = tp / (tp + fn)
    F = 2 * P * R / (P + R)
    print ('\nPrecision: {0}\t Recall: {1}\t F1-Score: {2}'.format(P, R, F))
    return {'P': P, 'R': R, 'F': F}
    
###new added file
def oe_score(y_true, y_pred):
    oe_no=0
    nt=len(y_pred)
    for i in range(nt):
        if abs(y_pred[i]-y_true[i])>1:
            oe_no+=1
    OE= oe_no/nt
    print('OE:{}'.format(OE))
    return {'OE': OE}  
def scores(ypred,ytest):
    ypred = np.argmax(ypred, axis=-1)
    ytest = np.argmax(ytest, axis=-1)
    score = get_prf1_score(ytest, ypred)
    oe=oe_score(ytest, ypred)
    return(score,oe)

**Training, Testing and Evaluation**

In [13]:
if __name__ == '__main__':
    ###begin time:
    start_time=time.time()

    with open(op_file, 'w') as of:

        x_data, y_data = read_data(ip_txt_file)

        ext_feature = read_external_features(ip_txt_file, ip_feat_file)

        cv_count = 0
        k_score = []
        oescore = []
       
        kscore_svmln=[]
        oescore_svmln=[]
        
        # Stratified cross-validation
        skf = StratifiedKFold(n_splits=sys_params['cross_val'])
        skf.get_n_splits(x_data, y_data)

        # Run the model for each splits
        for train_index, test_index in skf.split(x_data, y_data):
            cv_count += 1
            print ('\nRunning Stratified Cross Validation: {0}/{1}...'.format(cv_count, sys_params['cross_val']))

            x_train, x_test = x_data[train_index], x_data[test_index]
            y_train, y_test = y_data[train_index], y_data[test_index]

            # Convert the class labels into categorical
            y_train, y_test = to_categorical(y_train), to_categorical(y_test)

            # Reshape the data for CNN
            x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
            x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

            # External features for this particular split
            train_ext_feat, test_ext_feat = ext_feature[train_index], ext_feature[test_index]

            # CNN model for training on the embedded text input
            cnn_model = get_cnn_model()
            print (cnn_model.summary())

            # Train the model
            cnn_model.fit(x=x_train, y=y_train, batch_size=cnn_params['batch'], epochs=cnn_params['epoch'], verbose=cnn_params['verbose'])
            
            # Trained model for extracting features from intermediate layer
            model_feat_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer(intermediate_layer).output)

            # Get CNN gerated features
            train_cnn_feat = model_feat_extractor.predict(x_train)
            test_cnn_feat = model_feat_extractor.predict(x_test)

            # Merge the CNN generated features with the external features
            x_train_features = []
            for index, cnn_feature in enumerate(train_cnn_feat):
                tmp_feat = list(cnn_feature)
                ###When run model without these 14 features, comment the following lines. So these 14 features won't be added to the input data
                tmp_feat.extend(list(train_ext_feat[index]))
                x_train_features.append(np.array(tmp_feat))

            x_test_features = []
            for index, cnn_feature in enumerate(test_cnn_feat):
                tmp_feat = list(cnn_feature)
                ###When run model without these 14 features, comment the following lines. So these 14 features won't be added to the input data
                tmp_feat.extend(list(test_ext_feat[index]))
                x_test_features.append(np.array(tmp_feat))

            # Convert the list into numpy array
            x_train_features = np.array(x_train_features)
            x_test_features = np.array(x_test_features)

            del train_cnn_feat, test_cnn_feat

            # Get the MLP model for final classification
            mlp_model = get_mlp_model(ip_dim = len(x_train_features[0]))
            print (mlp_model.summary())

            tc = time.time()

            # Train the MLP model
            mlp_model.fit(x=x_train_features, y=y_train, batch_size=cnn_params['batch'], epochs=cnn_params['epoch'], verbose=cnn_params['verbose'])
                        
            print ('\nTime elapsed in training CNN: ', str(datetime.timedelta(seconds=time.time() - tc)))

            print ('\nEvaluating on Test data...\n')
            # # Print Loss and Accuracy
            model_metrics = mlp_model.evaluate(x_test_features, y_test)

            for i in range(len(model_metrics)):
                print (mlp_model.metrics_names[i], ': ', model_metrics[i])

            y_pred = mlp_model.predict(x_test_features)
            
            score,oe=scores(y_pred,y_test)
            k_score.append(score)
            oescore.append(oe)
            y_pred = np.argmax(y_pred, axis=-1)
            y_test_mlp = np.argmax(y_test, axis=-1)


            # Scikit-learn classification report (P, R, F1, Support)
            report = classification_report(y_test_mlp, y_pred)
            print (report)

            of.write('Cross_Val:\n')
            for i in range(len(y_pred)):
                of.write('\t'.join([str(y_test_mlp[i]), str(y_pred[i])]) + '\n')
           
            ###SVM model--linear
            ytrain_new=[key for ss in y_train for key,val in enumerate(ss) if val==1]
            ytest_new=[key for ss in y_test for key,val in enumerate(ss) if val==1]

            from sklearn.svm import SVC
            model_svm_ln = SVC(kernel='linear', probability=True)
            model_svm_ln.fit(x_train_features, np.array(ytrain_new))
            y_pred_svmln=model_svm_ln.predict(x_test_features)
            score_svmln = get_prf1_score(ytest_new, y_pred_svmln.tolist())
            oe_svmln=oe_score(ytest_new, y_pred_svmln.tolist())
            kscore_svmln.append(score_svmln)
            oescore_svmln.append(oe_svmln)       
            del x_train, y_train


        print('=============5 Labels MLP model: 300 Features from CNN model + 14 External Features\n')
#         print('=============CNN model: 300 Features\n')
        print (k_score)        
        print("\n",oescore)

        avgP = np.average([score['P'] for score in k_score])
        avgR = np.average([score['R'] for score in k_score])
        avgF = np.average([score['F'] for score in k_score])
        avgOE= np.average([score['OE'] for score in oescore])

        print ('\nAfter Stratified Cross Validation Average Precision: {0}\t Recall: {1}\t F1-Score: {2}\t OE-Score:{3}'.format(avgP, avgR, avgF,avgOE))
        
        print('=============5 Labels SVM Linear: 300 Features from CNN model + 14 External Features\n')

        print (k_score)        
        print("\n",oescore)

        avgP_svmln = np.average([score['P'] for score in kscore_svmln])
        avgR_svmln = np.average([score['R'] for score in kscore_svmln])
        avgF_svmln = np.average([score['F'] for score in kscore_svmln])
        avgOEs_svmln = np.average([score['OE'] for score in oescore_svmln])

        print ('\nAfter Stratified Cross Validation Average Precision: {0}\t Recall: {1}\t F1-Score: {2}\t OE-Score:{3}'.format(avgP_svmln,avgR_svmln,avgF_svmln,avgOEs_svmln ))
        end_time=time.time()
        print("Time Completing the tast:{} hours".format((end_time-start_time)/3600))


Running Stratified Cross Validation: 1/5...
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1500, 300,   0           []                               
                                1)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 1500, 1, 100  90100       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 1500, 1, 100  120100      ['input_1[0][0]']                
                                )                

2023-04-19 23:24:13.152824: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 720000000 exceeds 10% of free system memory.


Epoch 1/5
100/100 [==============================] - 41s 391ms/step - loss: 1.7391 - accuracy: 0.2875
Epoch 2/5
100/100 [==============================] - 37s 367ms/step - loss: 1.3335 - accuracy: 0.4950
Epoch 3/5
100/100 [==============================] - 39s 392ms/step - loss: 0.9221 - accuracy: 0.6500
Epoch 4/5
100/100 [==============================] - 37s 369ms/step - loss: 0.3868 - accuracy: 0.8975
Epoch 5/5
100/100 [==============================] - 39s 393ms/step - loss: 0.1585 - accuracy: 0.9850


2023-04-19 23:27:37.635838: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 720000000 exceeds 10% of free system memory.


4/4 [==============================] - 3s 771ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 classif_op (Dense)          (None, 5)                 1580      
                                                                 
Total params: 1,580
Trainable params: 1,580
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
100/100 [==============================] - 1s 2ms/step - loss: 116.8798 - accuracy: 0.1925
Epoch 2/5
100/100 [==============================] - 0s 2ms/step - loss: 20.4257 - accuracy: 0.2775
Epoch 3/5
100/100 [==============================] - 0s 2ms/step - loss: 8.3780 - accuracy: 0.4675
Epoch 4/5
100/100 [==============================] - 0s 2ms/step - loss: 7.6160 - accuracy: 0.4925
Epoch 5/5
100/100 [==============================] - 0s 2ms/step - loss: 6.6660 - accuracy: 0.5575

Time elapsed in training 

...

100/100 [==============================] - 39s 379ms/step - loss: 1.8092 - accuracy: 0.2875
Epoch 2/5
100/100 [==============================] - 39s 386ms/step - loss: 1.3659 - accuracy: 0.4500
Epoch 3/5
100/100 [==============================] - 39s 392ms/step - loss: 0.8620 - accuracy: 0.6850
Epoch 4/5
100/100 [==============================] - 37s 370ms/step - loss: 0.5482 - accuracy: 0.8125
Epoch 5/5
100/100 [==============================] - 39s 392ms/step - loss: 0.1992 - accuracy: 0.9550


2023-04-19 23:31:36.619568: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 720000000 exceeds 10% of free system memory.


4/4 [==============================] - 3s 830ms/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 classif_op (Dense)          (None, 5)                 1580      
                                                                 
Total params: 1,580
Trainable params: 1,580
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
100/100 [==============================] - 1s 2ms/step - loss: 44.8311 - accuracy: 0.2475
Epoch 2/5
100/100 [==============================] - 0s 2ms/step - loss: 12.6748 - accuracy: 0.3775
Epoch 3/5
100/100 [==============================] - 0s 2ms/step - loss: 15.0109 - accuracy: 0.4100
Epoch 4/5
100/100 [==============================] - 0s 2ms/step - loss: 8.9015 - accuracy: 0.5000
Epoch 5/5
100/100 [==============================] - 0s 2ms/step - loss: 6.0931 - accuracy: 0.6275

Time elapsed in trainin

2023-04-19 23:32:22.384491: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 720000000 exceeds 10% of free system memory.


100/100 [==============================] - 39s 379ms/step - loss: 1.6998 - accuracy: 0.3125
Epoch 2/5
100/100 [==============================] - 38s 376ms/step - loss: 1.4193 - accuracy: 0.4200
Epoch 3/5
100/100 [==============================] - 44s 440ms/step - loss: 0.9817 - accuracy: 0.6250
Epoch 4/5
100/100 [==============================] - 37s 372ms/step - loss: 0.4608 - accuracy: 0.8725
Epoch 5/5
4/4 [==============================] - 2s 430ms/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 classif_op (Dense)          (None, 5)                 1580      
                                                                 
Total params: 1,580
Trainable params: 1,580
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
100/100 [==============================] - 1s 2ms/step - loss: 36.9864 - accuracy: 0.3100
Epoch 2/5
100/